In [1]:
import os
import numpy as np
import pandas as pd
import re

In [13]:
file = '../pwscf.in'
f = open(file,"r")
file_lines = f.readlines() 

In [11]:
def file_len(fname):
    """count total number of lines in file"""
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def remove_empty_lines(fname):
    """removes empty lines and \n in a file"""
    if not os.path.isfile(fname):
        print("{} does not exist ".format(fname))
        return
    with open(fname) as filehandle:
        lines = filehandle.readlines()
    with open(fname, 'w') as filehandle:
        lines = filter(lambda x: x.strip() and x.strip('\n'), lines)
        filehandle.writelines(lines)

flatten = lambda l: [item for sublist in l for item in sublist]

def GetHeader(infile):
    param = []
    with open(infile) as fp:
        for groups in re.findall('&(.*?)/', fp.read(), re.S):
            param.append(groups.splitlines()[1:])
        return flatten(param)

def GetCell(infile):
    with open(infile) as fp:
        result = re.search(r'.*CELL_PARAMETERS\s*(.*)$', fp.read(), re.DOTALL)
    return result.group(1).splitlines()[1:]

def GetAtomPos(infile):
    with open(infile) as fp:
        result = re.findall(r'ATOMIC_POSITIONS(.*?)K_POINTS', fp.read(), re.S)
        return result
    
def slicefile(keyword):
    """find lines after keyword between & and /"""
    lines = re.findall('&%s(?:.?)+\n((?:.+\n)+?)(?:\s+)?[\/&]'%keyword,"".join(file_lines),re.MULTILINE)
    return lines

def store(group,name):
    """
    Save the variables specified in each of the groups on the structure
    """
    for file_slice in slicefile(name):
        for keyword, value in re.findall('([a-zA-Z_0-9_\(\)]+)(?:\s+)?=(?:\s+)?([a-zA-Z\'"0-9_.-]+)',file_slice):
            group[keyword.strip()]=value.strip()

In [359]:
remove_empty_lines(file)

In [102]:
file_len(file)

188

In [7]:
param = GetHeader(file);param

["    calculation = 'scf'",
 "    prefix = 'gc'",
 "    pseudo_dir = '",
 '    ibrav = 0',
 '    nat = 148',
 '    ntyp = 3',
 '    ecutwfc = 80',
 "    occupations = 'smearing'",
 "    smearing = 'gaussian'",
 '    degauss = 0.1',
 "    vdw_corr='grimme-d2'",
 '    tot_charge=1',
 "    diagonalization = 'david'",
 '    diago_thr_init = 1.d-4',
 "    mixing_mode = 'local-TF'",
 '    mixing_beta = 0.7',
 '    conv_thr =  1.d-8']

In [343]:
df_head = pd.DataFrame(param)
df_head_split = df_head.iloc[:,0].str.split("=", expand = True)
df_head_indexed = df_head_split.set_index([0])
df_head_indexed

,1
0,
calculation,'scf'
prefix,'gc'
pseudo_dir,'
ibrav,0
nat,148
ntyp,3
ecutwfc,80
occupations,'smearing'
smearing,'gaussian'


In [327]:
cell = GetCell(file); cell

['7.378073983 0.000000000 0.0000000000',
 '0.000000000 17.038932 0.0000000000',
 '0.000000000 0.000000000 25.00']

In [443]:
df_cell = pd.DataFrame(cell)
df_cell = df_cell[0].str.split(" ", expand = True)
df_cell.rename(index = {0:'a',1:'b',2:'c'},columns={0:'va',1:'vb',2:'vc'})

,va,vb,vc
a,7.378073983,0.000000000,0.0000000000
b,0.000000000,17.038932,0.0000000000
c,0.000000000,0.000000000,25.00


In [452]:
atoms = GetAtomPos(file)

In [449]:
df_atoms = pd.DataFrame(atoms)
df2 = df_atoms[0].str.split("\n", expand = True).T.drop(0)
df2_atoms = df2[0].replace('\s+', ' ', regex=True).str.split(" ",expand = True).dropna()
df2_atoms

,0,1,2,3
1,Cu,0.000000000,0.000418475,7.356821029
2,Cu,1.844464273,1.704158296,7.358113474
3,Cu,3.689036991,0.000230648,7.357039041
4,Cu,5.533609710,1.704158296,7.358113474
5,Cu,0.000000000,3.407517063,7.358466518
6,Cu,1.844400743,5.111308876,7.357351579
7,Cu,3.689036991,3.407727228,7.358240984
8,Cu,5.533673240,5.111308876,7.357351579
9,Cu,0.000000000,6.815511010,7.356398114
10,Cu,1.844634168,8.519772057,7.356953132


In [14]:
store(file,"control")

TypeError: 'str' object does not support item assignment

In [456]:
!ls {PATH2}

../pwscf.out
